In [0]:
#How to read the data in a batch mode instead of streaming
# This code will not accommodate new records this is not streaming
confluentBootstrapServers = 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092'
confluentApiKey = ''
confluentSecret = ''
confluentTopicName = 'retail-data-new'


In [0]:
#boiler plate code
orders_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "confluentBootstrapServers") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.mechanism","PLAIN") \
    .option("kafka.sasl.jaas.config","kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username= '{}' password='{}';".format(confluentApiKey, confluentSecret)) \
    .option("kafka.ssl.endpoint.identification.algorithm","https") \
    .option("subscribe", confluentTopicName) \
    # Run from very start time
    .option("startingTimestamp",1) \ 
    # Just to make sure all the micro batches are doing same amount of work
    .option("maxOffsetsPerTrigger",50) \
    .load()


In [0]:
display(orders_df)

In [0]:
converted_orders_df = orders_df.selectExpr("CAST(key as string) AS key","CAST(value as string) AS value","topic", "partition","offset","timestamp", "timestampType")

In [0]:
display(converted_orders_df)

In [0]:
converted_orders_df.writeStream
          .queryName("ingestionquery") \
          .format("delta") \
          .outputMode("append") \
          .option("checkpointLocation", "checkpointdir301") \
          .toTable("ordersnewtable301") 

In [0]:
spark.sql("select * from ordersnewtable301").show()